In [ ]:
import os
import sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pickle
import seaborn as sns
import json
import copy
import logging
from IPython.display import Markdown, display
logging.basicConfig(level=logging.INFO)
logging.getLogger("matplotlib").setLevel(logging.INFO)
logger = logging.getLogger(__name__)

sys.path.append("..")
import pandas as pd
from util.distance import dist3
from util.scenario import Scenario

box_color = '#86afb8'

In [ ]:
def min_distance(ndrones):
    """
    Returns the minimum travel distance for each UAV from the Villalvernia scenario
    """
    flightseqs_fpath = f"../out/flight_sequences/villalvernia_{ndrones}/flight_sequences.pkl"
    with open(flightseqs_fpath, 'rb') as f:
        flightseqs = pickle.load(f)   
    sc = Scenario([], positions_w=flightseqs)
    
    res = {}
    for d in range(sc.N_d):
        res[d] = sc.D_N[d].sum()
    return res

def sum_distance_from_dir(eventsdir):
    """
    Returns the collective distance from the events directory for all UAVs
    """
    res = {}
    for d, fname in enumerate(os.listdir(eventsdir)):
        fpath = os.path.join(eventsdir, fname)
        df = pd.read_csv(fpath)
        res[d] = sum_distance_from_df(df)
    return res

def sum_distance_from_df(df):
    """
    Returns the collective distance between all nodes visited in the event dataframe
    """
    res = 0
    for i in range(df.shape[0]-1):
        a = df.iloc[i][['node_x', 'node_y', 'node_z']]
        b = df.iloc[i+1][['node_x', 'node_y', 'node_z']]
        dist = dist3(a,b)
        res += dist
    return res

In [ ]:
def get_nr_charge_cycles(loaded):
    nr_drones = loaded['scenario']['nr_drones']

    res = []
    for d in range(nr_drones):
        charge_cycles = 0
        currently_charging = False
        for ev in loaded['events'][d]:
            if ev['type'] == 'charged':
                currently_charging = True            
            elif currently_charging: 
                # stop charging            
                charge_cycles += 1
                currently_charging = False
        res.append(charge_cycles)
    return res
    

In [ ]:
def load_data_from_json(basedir):
    data = []
    for subdir in os.listdir(basedir):
        if os.path.isdir(os.path.join(basedir, subdir)):
            result_fpath = os.path.join(basedir, subdir, "result.json")
            if not os.path.exists(result_fpath):
                logger.info(f"skipping '{subdir}' due to missing 'result.json' file")
                continue                
            
            with open(result_fpath, 'r') as f:
                try:
                    loaded = json.load(f)
                except Exception as e:
                    logger.error(f"failed to load JSON for '{subdir}': {e}")
                    continue
                
                datapoint = {}

                # parse parameters
                params = loaded['params']
                
                cols = ['v', 'r_charge', 'r_deplete', 'B_max', 'B_min', 'epsilon', 'schedule_delta', 'W', 'sigma', 'time_limit', 'int_feas_tol', 'rescheduling_frequency']
                for col in cols:
                    val = params[col]
                    if type(val) == list:
                        val = val[0]
                    datapoint[col] = val              
                    
                # parse relevant results
                datapoint['scheduler'] = loaded['scheduler']
                datapoint['dirname'] = subdir
                datapoint['execution_time'] = loaded['execution_time']
                datapoint['nr_drones'] = loaded['scenario']['nr_drones']
                datapoint['solve_times'] = [x['t_solve'] for x in loaded['solve_times']]
                datapoint['solve_timestamps'] = [x['timestamp'] for x in loaded['solve_times']]
                datapoint['solve_optimality'] = [x['optimal'] for x in loaded['solve_times']]
                datapoint['moving_times'] = [x['moving'] for x in loaded['time_spent'].values()]
                datapoint['waiting_times'] = [x['waiting'] for x in loaded['time_spent'].values()]
                datapoint['charging_times'] = [x['charging'] for x in loaded['time_spent'].values()]
                datapoint['moving_times_minimum'] = [x['moving_minimum'] for x in loaded['time_spent'].values()]
                datapoint['total_occupancy'] = [sum(x['t_end'] - x['t_start'] for x in l) for l in loaded['occupancy'].values()]
                datapoint['nr_charge_cycles'] = get_nr_charge_cycles(loaded)
                
                data.append(datapoint)
    res = pd.DataFrame(data)
    
    # preprocessing
    # basic statistical functions
    cols = ['moving_times', 'waiting_times', 'charging_times', 'solve_times', 'nr_charge_cycles']
    for col in cols:
        new_col = f"{col}_cum"
        res[new_col] = res[col].apply(sum)
        
        new_col = f"{col}_mean"
        res[new_col] = res[col].apply(np.mean)
        
        new_col = f"{col}_min"
        res[new_col] = res[col].apply(np.min)
        
        new_col = f"{col}_max"
        res[new_col] = res[col].apply(np.max)

    # counts
    cols = ['solve_times']
    for col in cols:
        new_col = f"{col}_count"
        res[new_col] = res[col].apply(len)
        
    # percent spent on different actions
    cols = ['moving_times', 'waiting_times', 'charging_times']
    total = res[cols].apply(lambda x: sum([np.array(v) for v in x]), axis=1)
    for col in cols:
        new_col = f"{col}_perc"
        res[new_col] = res[col].apply(np.array) / total * 100
        
        new_mean_col = f"{col}_perc_mean"
        res[new_mean_col] = res[new_col].apply(np.mean)
        
    res['perc_occupancy'] = res.apply(lambda x: np.array(x['total_occupancy']) / x['execution_time'] * 100, axis=1)    
    res['solve_optimality_perc'] = res.solve_optimality.apply(lambda x: np.sum(x) / len(x) * 100)
    
    # strings
    res['scheduler'] = res.scheduler.apply(lambda x: x[:-9])
    
    # column renaming
    res = res.rename(columns={'rescheduling_frequency': 'pi'})
    
    res['h'] = (res.W - 1) * res.sigma
    res['perc_max_horizon_executed'] = res['pi'] / res['h'] * 100
    res['solve_times_optimal_mean'] = res.solve_times.apply(lambda x: np.mean([e for e in x if e < 10]))
    
    # TOOD: extract nr charging cycles
    
    return res

# Planning horizon

In [ ]:
basedir = "../out/villalvernia/grid_search"
df_planning_horizon = load_data_from_json(basedir)
# df_planning_horizon = df_planning_horizon[lambda x: x.h <= 160]
df_planning_horizon_milp = df_planning_horizon[lambda x: x.scheduler == 'milp']
df_planning_horizon_naive = df_planning_horizon[lambda x: x.scheduler == 'naive']
naive_execution_time = df_planning_horizon_naive.iloc[0].execution_time
df_planning_horizon_milp['perc_execution_time_naive'] = df_planning_horizon_milp.execution_time / naive_execution_time * 100

logger.info(f"Number of data points: {df_planning_horizon.shape[0]}")

In [ ]:
Ws = sorted(df_planning_horizon_milp.W.unique())
baseline_execution_time = df_planning_horizon_naive.iloc[0].execution_time
max_execution_time = df_planning_horizon_milp.execution_time.max()
max_time_limit = df_planning_horizon_milp.time_limit.max()
min_perc_execution_time_naive = df_planning_horizon_milp.perc_execution_time_naive.min()
max_perc_execution_time_naive = df_planning_horizon_milp.perc_execution_time_naive.max()
min_solve_times_count = df_planning_horizon_milp.solve_times_count.min()
max_solve_times_count = df_planning_horizon_milp.solve_times_count.max()
min_solve_times_cum = df_planning_horizon_milp.solve_times_cum.min()
max_solve_times_cum = df_planning_horizon_milp.solve_times_cum.max()

min_moving_times_mean = df_planning_horizon_milp.moving_times_mean.min()
max_moving_times_mean = df_planning_horizon_milp.moving_times_mean.max()
min_waiting_times_mean = df_planning_horizon_milp.waiting_times_mean.min()
max_waiting_times_mean = df_planning_horizon_milp.waiting_times_mean.max()
min_charging_times_mean = df_planning_horizon_milp.charging_times_mean.min()
max_charging_times_mean = df_planning_horizon_milp.charging_times_mean.max()

min_moving_times_perc_mean = df_planning_horizon_milp.moving_times_perc_mean.min()
max_moving_times_perc_mean = df_planning_horizon_milp.moving_times_perc_mean.max()
min_waiting_times_perc_mean = df_planning_horizon_milp.waiting_times_perc_mean.min()
max_waiting_times_perc_mean = df_planning_horizon_milp.waiting_times_perc_mean.max()
min_charging_times_perc_mean = df_planning_horizon_milp.charging_times_perc_mean.min()
max_charging_times_perc_mean = df_planning_horizon_milp.charging_times_perc_mean.max()

nr_cols = 17
_, axes = plt.subplots(len(Ws), nr_cols, dpi=100, figsize=(6*nr_cols, 5*len(Ws)))

for i, W in enumerate(Ws):    
    # plot execution time
    axidx = 0
    rectangles = []
    
    ax = axes[i][axidx]
    df_plot_et = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'execution_time')
    sns.heatmap(df_plot_et / 60, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=",.0f", cbar=False, linewidths=3, vmin=0, vmax=max_execution_time / 60, ax=ax) 
    for r in range(df_plot_et.shape[0]):
        for c in range(df_plot_et.shape[1]):
            if df_plot_et.iloc[r,c] < baseline_execution_time:
                # draw green rectangle around box
                rectangles.append(patches.Rectangle((c+0.025, r+0.025), 0.95, 0.95, linestyle="-", linewidth=2, edgecolor='g', facecolor='none', alpha=0.25))
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Execution time (m) (W={W})")
    axidx += 1
    
    # plot relative execution time compared to Naive
    ax = axes[i][axidx]
    df_plot_pet = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'perc_execution_time_naive')
    sns.heatmap(df_plot_pet, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".0f", cbar=False, linewidths=3, vmin=min_perc_execution_time_naive, vmax=max_perc_execution_time_naive, ax=ax)
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Relative execution time (%) (W={W})")
    axidx += 1
    
    # plot mean solve times
    ax = axes[i][axidx]
    df_plot_mst = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'solve_times_mean')
    sns.heatmap(df_plot_mst, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=0, vmax=max_time_limit, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Mean solve time (s) (W={W})")
    axidx += 1
    
    # plot mean optimal solve times
    ax = axes[i][axidx]
    df_plot_mst = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'solve_times_optimal_mean')
    sns.heatmap(df_plot_mst, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=0, vmax=max_time_limit, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Mean optimal solve time (s) (W={W})")
    axidx += 1
           
    # plot solve_time_count
    ax = axes[i][axidx]
    df_plot_stc = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'solve_times_count')
    sns.heatmap(df_plot_stc, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".0f", cbar=False, linewidths=3, vmin=min_solve_times_count, vmax=max_solve_times_count, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Solve time count (W={W})")
    axidx += 1
    
    # plot sum solve times
    ax = axes[i][axidx]
    df_plot_cst = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'solve_times_cum')
    sns.heatmap(df_plot_cst, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".0f", cbar=False, linewidths=3, vmin=min_solve_times_cum, vmax=max_solve_times_cum, ax=ax)
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Sum solve time (s) (W={W})")
    axidx += 1
    
    # plot percentage of solves optimal
    ax = axes[i][axidx]
    df_plot_pst = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'solve_optimality_perc')
    sns.heatmap(df_plot_pst, robust=True, square=True, cmap='Blues_r', annot=True, annot_kws={"fontsize":8}, fmt=".0f", cbar=False, linewidths=3, vmin=0, vmax=100, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Optimal solves (%) (W={W})")
    axidx += 1
    
    # perc occupancy (station 1)
    ax = axes[i][axidx]
    df_plot_po1 = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'perc_occupancy').apply(lambda x: [v[0] if type(v) == np.ndarray else v for v in x]) 
    sns.heatmap(df_plot_po1, robust=True, square=True, cmap='Blues_r', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=0, vmax=100, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"CS [1] occupancy (%) (W={W})")
    axidx += 1    
    
    # perc occupancy (station 2)
    ax = axes[i][axidx]
    df_plot_po1 = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'perc_occupancy').apply(lambda x: [v[1] if type(v) == np.ndarray else v for v in x]) 
    sns.heatmap(df_plot_po1, robust=True, square=True, cmap='Blues_r', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=0, vmax=100, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"CS [2] occupancy (%) (W={W})")
    axidx += 1
    
    # moving time mean
    ax = axes[i][axidx]
    df_plot_mwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'moving_times_mean')
    sns.heatmap(df_plot_mwt / 60, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=min_moving_times_mean / 60, vmax=max_moving_times_mean / 60, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Mean moving time (m) (W={W})")
    axidx += 1 
    
    # waiting time mean
    ax = axes[i][axidx]
    df_plot_wwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'waiting_times_mean')
    sns.heatmap(df_plot_wwt / 60, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=min_waiting_times_mean / 60, vmax=max_waiting_times_mean / 60, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Mean waiting time (m) (W={W})")
    axidx += 1 
    
    # charging time mean
    ax = axes[i][axidx]
    df_plot_cwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'charging_times_mean')
    sns.heatmap(df_plot_cwt / 60, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=min_charging_times_mean / 60, vmax=max_charging_times_mean / 60, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Mean charging time (m) (W={W})")
    axidx += 1 

    # moving time percentage mean
    ax = axes[i][axidx]
    df_plot_pwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'moving_times_perc_mean')
    sns.heatmap(df_plot_pwt, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=min_moving_times_perc_mean, vmax=max_moving_times_perc_mean, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"$\mu$ % time spent moving (W={W})")
    axidx += 1 
    
    # waiting time percentage mean
    ax = axes[i][axidx]
    df_plot_pwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'waiting_times_perc_mean')
    sns.heatmap(df_plot_pwt, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=min_waiting_times_perc_mean, vmax=max_waiting_times_perc_mean, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"$\mu$ % time spent waiting (W={W})")
    axidx += 1 
    
    # charging time percentage mean
    ax = axes[i][axidx]
    df_plot_pwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'charging_times_perc_mean')
    sns.heatmap(df_plot_pwt, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=min_charging_times_perc_mean, vmax=max_charging_times_perc_mean, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"$\mu$ % time spent charging (W={W})")
    axidx += 1 
    
    # percentage of scheduling horizon at maximum executed
    ax = axes[i][axidx]
    df_plot_pwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'perc_max_horizon_executed')
    sns.heatmap(df_plot_pwt, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=0, vmax=100, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Maximum % of horizon executed (W={W})")
    axidx += 1 
    
    # percentage of scheduling horizon at maximum executed
    ax = axes[i][axidx]
    df_plot_pwt = df_planning_horizon_milp[lambda x: x.W == W].pivot('sigma', 'pi', 'h')
    sns.heatmap(df_plot_pwt, robust=True, square=True, cmap='Blues', annot=True, annot_kws={"fontsize":8}, fmt=".1f", cbar=False, linewidths=3, vmin=0, vmax=100, ax=ax) 
    for rect in rectangles:
        cpy = copy.copy(rect)
        ax.add_patch(cpy)
    ax.set_xlabel("$\pi$")
    ax.set_ylabel("$\sigma$")
    ax.set_title(f"Scheduling horizon (W={W})")
    axidx += 1 

plt.savefig("../out/figures/heatmaps.pdf", bbox_inches='tight')
plt.show()

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)

df_filtered = df_increase_sigma_milp.drop_duplicates(['sigma'])
X = df_filtered.sigma
Y = df_filtered.execution_time
baseline_execution_time = df_increase_sigma[lambda x: x.schedule_type == 'naive'].iloc[0].execution_time

plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)
xmin = 0.1
xmax = 20.8
ax.set_xlim([xmin, xmax])
plt.axhline(baseline_execution_time, color='red')
plt.text(xmax-0.1, baseline_execution_time+100, "Naive strategy", c='red', ha='right', va='bottom', fontsize=9)

ax.set_axisbelow(False)
plt.grid(axis='y')
plt.xticks(X, fontsize=8)
plt.xlabel("$\sigma$")
plt.ylabel("Execution time (s)")

plt.savefig("../out/figures/1_planning_horizon_increase_sigma.pdf", bbox_inches='tight')
plt.show()

In [ ]:
# Calculate precise gain

In [ ]:
df_increase_sigma[lambda x: x.sigma > 7].execution_time.min()

In [ ]:
lowest_execution_time = df_increase_sigma[lambda x: x.sigma > 7].execution_time.min()
print(f"Lowest execution time = {lowest_execution_time:.1f}s")
print(f"                   or = {(1 - (lowest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

In [ ]:
highest_execution_time = df_increase_sigma[lambda x: x.sigma > 7].execution_time.max()
print(f"Highest execution time = {highest_execution_time:.1f}s")
print(f"                    or = {(1 - (highest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

## Increase $W$

In [ ]:
basedir = "../out/villalvernia/1_planning_horizon/increase_W"
df_full_coverage = load_data(basedir)
df_full_coverage_milp = df_full_coverage[lambda x: x.schedule_type == 'milp']
df_full_coverage_milp = df_full_coverage_milp[lambda x: x.W > 6]

In [ ]:
# Execution times

In [ ]:
X = df_full_coverage_milp.W
Y = df_full_coverage_milp.execution_time
baseline_execution_time = df_full_coverage[lambda x: x.schedule_type == 'naive'].iloc[0].execution_time

_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)

xmin = 6.1
xmax = 20.8
ax.set_xlim([xmin, xmax])
plt.axhline(baseline_execution_time, color='red')
plt.text(xmax-0.1, baseline_execution_time-50, "Naive strategy", c='red', ha='right', va='top', fontsize=9)

plt.grid(axis='y')
ax.set_xticks(X)
ax.set_xlabel("W")
ax.set_ylabel("Execution time (s)")

plt.savefig("../out/figures/1_planning_horizon_increase_W_execution_time.pdf", bbox_inches='tight')
plt.show()

In [ ]:
lowest_execution_time = df_full_coverage_milp.execution_time.min()
print(f"Lowest execution time = {lowest_execution_time:.1f}s")
print(f"                   or = {(1 - (lowest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

highest_execution_time = df_full_coverage_milp.execution_time.max()
print(f"Highest execution time = {highest_execution_time:.1f}s")
print(f"                    or = {(1 - (highest_execution_time / baseline_execution_time)) * 100:.1f}% faster than naive")

In [ ]:
# Solve times (total)

In [ ]:
X = sorted(df_full_coverage_milp.W.unique())
Y = df_full_coverage_milp.groupby('W').solve_time.sum()

_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)

plt.grid(axis='y')
xmin = 6.1
xmax = 20.8
ax.set_xlim([xmin, xmax])
ax.set_xticks(X)
ax.set_xlabel("W")
ax.set_ylabel("Total solve time (s)")

plt.savefig("../out/figures/1_planning_horizon_increase_W_solve_time_total.pdf", bbox_inches='tight')
plt.show()

In [ ]:
# Solve times (boxplot)

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
sns.boxplot(x="W", y="solve_time", data=df_full_coverage_milp, ax=ax, width=0.6, linewidth=1, whis=[0, 100], zorder=10, color=box_color)
plt.yscale('log')
plt.ylabel("Solve time [log]")
plt.xlabel("$\hat{N}_w$")

ticks = [1, 10, 60, 600]
labels = ['1s', '10s', '1m', '10m']
plt.yticks(ticks, labels)
ax.set_axisbelow(True)
plt.grid(axis='y')

# add red horizontal line
ax.axhline(y=600, c='red')
offsetx, offsety = 0.1, -100
ax.text(ax.get_xlim()[0]+offsetx, 600+offsety, 'Time limit', color='red', va='top', ha='left', fontsize=9)

plt.savefig("../out/figures/1_planning_horizon_increase_W_solve_time_box.pdf", bbox_inches='tight')
plt.show()

# Number of drones

In [ ]:
basedir = "../out/villalvernia/2_n_drones"
df_n_drones = load_data(basedir).drop_duplicates(['schedule_type', 'n_drones']).sort_values('schedule_type', ascending=True).sort_values('n_drones', ascending=False)
# df_n_drones = load_data(basedir)
df_n_drones_milp = df_n_drones[lambda x: (x.schedule_type == 'milp')]
df_n_drones_naive = df_n_drones[lambda x: x.schedule_type == 'naive']

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
df_plot = pd.concat([df_n_drones_milp, df_n_drones_naive])
sns.barplot(x='n_drones', y='execution_time', hue='schedule_type', data=df_plot, edgecolor='black', linewidth=0.75, hue_order=['naive', 'milp'])

# add gains
for i, n_drones in enumerate(sorted(df_plot.n_drones.unique())):
    milp_time = df_n_drones_milp[lambda x: x.n_drones == n_drones].iloc[0].execution_time
    naive_time = df_n_drones_naive[lambda x: x.n_drones == n_drones].iloc[0].execution_time
    max_time = max(milp_time, naive_time)
    rel_perc = (milp_time / naive_time * 100)
    diff = 100 - rel_perc

    if diff > 0:
        color = 'green'
        txt = r"$\downarrow$" + f"{-diff:.1f}%"
    else:
        color = 'red'
        txt = r"$\uparrow$" + f"+{-diff:.1f}%"
    plt.text(i, max_time + 50, txt, ha='center', va='bottom', backgroundcolor='white', color=color, fontsize=9, bbox=dict(boxstyle='square,pad=0.1', fc='white', ec='none'))
    
ymin, ymax = plt.ylim()
plt.ylim([ymin, ymax+500])
plt.legend(fontsize=8)
plt.xlabel("$N_d$")
plt.ylabel("Execution time (s)")

ax.set_axisbelow(True)
plt.grid(axis='y')

plt.savefig("../out/figures/2_n_drones.pdf", bbox_inches='tight')
plt.show()

# Time limit on MILP solving

In [ ]:
basedir = "../out/villalvernia/3_solve_time"
df_solve_time = load_data(basedir)
df_solve_time_milp = df_solve_time[lambda x: x.schedule_type == 'milp']

In [ ]:
df_latex = df_solve_time_milp[['time_limit', 'execution_time']].drop_duplicates().sort_values(by='time_limit')
df_latex['time_limit'] = df_latex['time_limit'].astype(int)
df_latex.columns = ['Time limit', 'Execution time']

print(df_latex.to_latex(index=False))

# Rescheduling policy

In [ ]:
basedir = "../out/villalvernia/4_rescheduling_policy"
df_rescheduling_policy = load_data(basedir)
df_rescheduling_policy_milp = df_rescheduling_policy[lambda x: x.schedule_type == 'milp']

In [ ]:
df_rescheduling_policy_milp.drop_duplicates('rescheduling_frequency').sort_values(by='rescheduling_frequency')

In [ ]:
labels = df_rescheduling_policy_milp.sort_values(by='rescheduling_frequency').drop_duplicates('rescheduling_frequency').rescheduling_frequency.astype(int)
X = range(len(labels))
Y = df_rescheduling_policy_milp.sort_values(by='rescheduling_frequency').drop_duplicates('rescheduling_frequency').execution_time

_, ax = plt.subplots(figsize=np.array((4.5,2))*0.65, dpi=100)
plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)
plt.xticks(ticks=X, labels=labels)
plt.xlabel("Rescheduling frequency")
plt.ylabel("Execution\ntime (s)")

plt.savefig("../out/figures/4_rescheduling_policy_execution_time.pdf", bbox_inches='tight')
plt.show()

In [ ]:
labels = df_rescheduling_policy_milp.groupby('rescheduling_frequency').solve_time.sum().index 
X = range(len(labels))
Y = df_rescheduling_policy_milp.groupby('rescheduling_frequency').solve_time.sum()

_, ax = plt.subplots(figsize=np.array((4.5,2))*0.65, dpi=100)
plt.bar(X, Y, linewidth=.8, edgecolor='black', color=box_color, width=0.7)
plt.gca().invert_yaxis()
plt.ylabel("Solving time\n[sum] (s)")
plt.xticks([])

plt.savefig("../out/figures/4_rescheduling_policy_solving_time_sum.pdf", bbox_inches='tight')
plt.show()

# Charging/depletion ratio

In [ ]:
basedir = "../out/villalvernia/5_charging_rate"
df_charging_rate = load_data(basedir).drop_duplicates(['schedule_type', 'r_charge']).sort_values('schedule_type', ascending=True).sort_values('r_charge', ascending=True)
df_charging_rate['r_ratio'] = (df_charging_rate.r_charge / df_charging_rate.r_deplete).round(4)
df_charging_rate_milp = df_charging_rate[lambda x: x.schedule_type == 'milp']
df_charging_rate_naive = df_charging_rate[lambda x: x.schedule_type == 'naive']

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
df_plot = pd.concat([df_charging_rate_naive, df_charging_rate_milp])
sns.barplot(data=df_plot, x='r_ratio', y='execution_time', hue='schedule_type', edgecolor='black', linewidth=0.75, hue_order=['naive', 'milp'])

# add gains
for i, r_ratio in enumerate(df_plot.r_ratio.unique()):
    milp_time = df_charging_rate_milp[lambda x: x.r_ratio == r_ratio].iloc[0].execution_time
    naive_time = df_charging_rate_naive[lambda x: x.r_ratio == r_ratio].iloc[0].execution_time
    max_time = max(milp_time, naive_time)
    rel_perc = (milp_time / naive_time * 100)
    diff = 100 - rel_perc

    if diff > 0:
        color = 'green'
        txt = r"$\downarrow$" + f"{-diff:.1f}%"
    else:
        color = 'red'
        txt = r"$\uparrow$" + f"+{-diff:.1f}%"
    plt.text(i, max_time + 50, txt, ha='center', va='bottom', backgroundcolor='white', color=color, fontsize=9, bbox=dict(boxstyle='square,pad=0.1', fc='white', ec='none'))

ymin, ymax = plt.ylim()
plt.ylim([ymin, ymax+500])
plt.legend(fontsize=8)
# plt.xticks(ticks=df_plot.x_val.unique(), labels=df_plot.r_ratio.unique())
plt.xlabel(r"Ratio charging rate and depletion rate")
plt.ylabel("Execution time (s)")

ax.set_axisbelow(True)
plt.grid(axis='y')

plt.savefig("../out/figures/5_charging_rate.pdf", bbox_inches='tight')
plt.show()

# Charging cycle

In [ ]:
basedir = "../out/villalvernia/6_charging_cycle"
df_charge_cycle = load_data(basedir).drop_duplicates(['schedule_type', 'B_min'])
df_charge_cycle_milp = df_charge_cycle[lambda x: x.schedule_type == 'milp']
df_charge_cycle_naive = df_charge_cycle[lambda x: x.schedule_type == 'naive']

In [ ]:
_, ax = plt.subplots(figsize=np.array((4.5,2))*1, dpi=100)
sns.barplot(data=df_charge_cycle, x='B_min', y='execution_time', hue='schedule_type', edgecolor='black', linewidth=0.75, hue_order=['naive', 'milp'])


# add gains
for i, B_min in enumerate(sorted(df_charge_cycle.B_min.unique())):
    milp_time = df_charge_cycle_milp[lambda x: x.B_min == B_min].iloc[0].execution_time
    naive_time = df_charge_cycle_naive[lambda x: x.B_min == B_min].iloc[0].execution_time
    max_time = max(milp_time, naive_time)
    rel_perc = (milp_time / naive_time * 100)
    diff = 100 - rel_perc

    if diff > 0:
        color = 'green'
        txt = r"$\downarrow$" + f"{-diff:.1f}%"
    else:
        color = 'red'
        txt = r"$\uparrow$" + f"+{-diff:.1f}%"
    plt.text(i, max_time + 50, txt, ha='center', va='bottom', backgroundcolor='white', color=color, fontsize=9, bbox=dict(boxstyle='square,pad=0.1', fc='white', ec='none'))
    
ymin, ymax = plt.ylim()
plt.ylim([ymin, ymax+500])
plt.legend(fontsize=8, loc='lower right')
plt.xlabel("$B^{min}$")
plt.ylabel("Execution time (s)")

ax.set_axisbelow(True)
plt.grid(axis='y')

plt.savefig("../out/figures/6_charging_cycle.pdf", bbox_inches='tight')
plt.show()